In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics

from keras.models import Sequential
from keras.layers import LSTM,Dense,Flatten,Dropout

df = pd.read_json("bonus task regression\ipt_12.json")
df.head()

,case,run,VB,DOC,time,material,feed,smcAC,smcDC,vib_table,vib_spindle,AE_table,AE_spindle
0,1,1,0.00,1.5,2,1,0.5,"[-0.0170898438, 0.263671875, 0.2075195312, 0.3...","[0.625, 0.810546875, 0.7812500000000001, 0.849...","[0.078125, 0.0854492188, 0.078125, 0.073242187...","[0.3149414062, 0.3015136719, 0.3039550781, 0.3...","[0.0872802734, 0.0982666016, 0.0921630859, 0.0...","[0.1037597656, 0.1232910156, 0.1049804688, 0.1..."
1,1,2,NaN,1.5,4,1,0.5,"[0.3076171875, 0.3125, 0.1953125, 0.1879882812...","[0.6689453125, 0.6787109375, 0.859375, 1.12304...","[0.0756835938, 0.08056640620000001, 0.078125, ...","[0.3015136719, 0.3088378906, 0.2990722656, 0.3...","[0.0866699219, 0.0897216797, 0.0946044922, 0.1...","[0.0994873047, 0.1037597656, 0.1080322266, 0.1..."
2,1,3,NaN,1.5,6,1,0.5,"[-0.7250976562, -0.7153320312, -0.5297851562, ...","[0.9130859375, 0.8349609375, 0.908203125000000...","[0.0830078125, 0.078125, 0.09033203120000001, ...","[0.2954101562, 0.2966308594, 0.29296875, 0.290...","[0.0927734375, 0.1007080078, 0.0933837891, 0.0...","[0.1049804688, 0.1190185547, 0.109863281200000..."
3,1,4,0.11,1.5,7,1,0.5,"[0.1123046875, 0.009765625, -0.1513671875, -0....","[0.1318359375, 0.3955078125, 0.7568359375, 0.8...","[0.0830078125, 0.0756835938, 0.0659179688, 0.0...","[0.3161621094, 0.3112792969, 0.302734375, 0.32...","[0.1129150391, 0.0994873047, 0.1049804688, 0.1...","[0.1397705078, 0.12145996090000001, 0.12451171..."
4,1,5,NaN,1.5,11,1,0.5,"[-0.1220703125, -0.1635742188, -0.3686523438, ...","[0.44921875, 0.6640625, 0.6689453125, 0.747070...","[0.107421875, 0.107421875, 0.1025390625, 0.100...","[0.2844238281, 0.2893066406, 0.2844238281, 0.2...","[0.0958251953, 0.09765625, 0.09765625, 0.09460...","[0.1104736328, 0.1135253906, 0.109863281200000..."


In [2]:
print(df.shape)
print(df.columns)

for col in df.columns:
    if df[col].dtypes == 'object':
        count = 0
        for val in df[col]:
            nan_sum = 0
            nan_sum = np.isnan(val).sum()
            count += nan_sum
        print(col+": ", df[col].dtypes, ",", count)
    else:
        print(col+": ", df[col].dtypes, ",", df[col].isna().sum(axis=0))
        
for col in df.columns:
    if df[col].dtypes != 'object':
        print(col+": ", df[col].unique())
        
df.dropna(axis=0, how="any", inplace=True)
print(df.shape)
df = df[df.columns.drop("VB")]
print(df.shape)

(167, 13)
Index(['case', 'run', 'VB', 'DOC', 'time', 'material', 'feed', 'smcAC',
       'smcDC', 'vib_table', 'vib_spindle', 'AE_table', 'AE_spindle'],
      dtype='object')
case:  int64 , 0
run:  int64 , 0
VB:  float64 , 21
DOC:  float64 , 0
time:  int64 , 0
material:  int64 , 0
feed:  float64 , 0
smcAC:  object , 0
smcDC:  object , 0
vib_table:  object , 0
vib_spindle:  object , 0
AE_table:  object , 0
AE_spindle:  object , 0
case:  [ 1  2  3  4  9 10 11 12  5  6  7  8 13 14 15 16]
run:  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
VB:  [0.    nan 0.11 0.2  0.24 0.29 0.28 0.38 0.4  0.43 0.45 0.5  0.44 0.08
 0.14 0.15 0.18 0.22 0.26 0.31 0.48 0.55 0.13 0.17 0.19 0.23 0.33 0.36
 0.35 0.49 0.1  0.27 0.47 0.64 0.81 0.04 0.16 0.25 0.53 0.7  0.07 0.09
 0.12 0.37 0.42 0.57 0.65 0.68 0.76 0.05 0.32 0.58 0.74 0.34 0.46 0.3
 0.62 0.56 0.83 0.92 1.07 1.3  1.53 0.6  1.14]
DOC:  [1.5  0.75]
time:  [  2   4   6   7  11  15  19  22  26  29  32  35  38  41  44  46  48   3


In [42]:
for row in df.index:
    X_iter = []
    for i in range(9000):
        inter = []
        for sus in df.iloc[row]:
            print(sus)
            if isinstance(sus, list) is True:
                np.append(inter, sus[i])
            else:
                np.append(inter, sus)
            print(inter)
            break
        np.append(X_iter, inter)
        break
    break
print(X_iter)

1
[]
[]


In [38]:
# cases=[]
# runs=[]
# #VBs=[]
# DOCs=[]
# times=[]
# materials=[]
# feeds=[]
# smcACs=[]
# smcDCs=[]
# vib_tables=[]
# vib_spindles=[]
# AE_tables=[]
# AE_spindles=[]
# for i,row in df.iterrows():
#     case=row.case
#     run=row.run
#     #VB=row.VB
#     DOC=row.DOC
#     time=row.time
#     material=row.material
#     feed=row.feed
#     smcAC=row.smcAC
#     smcDC=row.smcDC
#     vib_table=row.vib_table
#     vib_spindle=row.vib_spindle
#     AE_table=row.AE_table
#     AE_spindle=row.AE_spindle
#     for m,ing in enumerate(row.smcAC):
#         cases.append(case)
#         runs.append(run)
#         #VBs.append(VB)
#         DOCs.append(DOC)
#         times.append(times)
#         materials.append(materials)
#         feeds.append(feeds)
#         smcACs=df["smcAC"].iloc[i][m]
#         smcDCs=df["smcDC"].iloc[i][m]
#         vib_tables=vib_table[0]
#         vib_spindles=vib_spindle[0]
#         AE_tables=AE_table[0]
#         AE_spindles=AE_spindle[0]
# itc=pd.DataFrame({"case":cases,"run":runs,"smcAC":smcACs})

IndexError: single positional indexer is out-of-bounds

In [ ]:
X = pd.DataFrame(df.dim_0.tolist())
X = X.to_numpy()

y = df["labels"].to_numpy(dtype=int)

In [ ]:
# def dimenstion_mod(x):
#     a = list()
#     for i in range(x.shape[0]):
#         a.append(x[i])
#     a = np.array(a) 
#     return np.reshape(a, (a.shape[0], a.shape[1],1))

In [ ]:
# X =  df[df.columns.drop("labels")].to_numpy()
# y = df["labels"].to_numpy(dtype=int)

y = np.where(y == -1, 0, y)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train)

# X_train = dimenstion_mod(X_train)
# X_val = dimenstion_mod(X_val)
# X_test = dimenstion_mod(X_test)

In [ ]:
model = Sequential()
model.add(LSTM(100,
            return_sequences=True,
            input_shape=(X_train.shape[1],1)))
model.add(LSTM(200, return_sequences=True))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
#model.add(LSTM(20))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss=tf.keras.losses.binary_crossentropy)

In [ ]:
history = model.fit(X_train,y_train,epochs=100, validation_data=(X_val, y_val), validation_steps = 20 ,batch_size=100)

y_pred = model.predict(X_test)
y_pred = np.where(y_pred > 0.5,1,0)

In [ ]:
cm = metrics.confusion_matrix(y_test, y_pred[:,0])
balanced_accuracy = metrics.balanced_accuracy_score(y_test, y_pred[:,0])
print(cm)
print(balanced_accuracy)